In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import pytorch_lightning as pl
import torch.optim as optim
from pytorch_lightning.loggers import TensorBoardLogger
import torchmetrics

In [14]:
train_epochs=512
learning_rate = 1e-5
weight_decay=1e-4
batch_size = 32

In [15]:
df = pd.read_csv('mixed.csv')

X = df.drop('gesture', axis=1).to_numpy()
y = df['gesture'].to_numpy()

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=114514)
# x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=114514)

In [16]:
class GestureDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32).to("cuda")
        self.y = torch.tensor(y, dtype=torch.long).to("cuda")

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# 训练集和验证集的 Dataset
train_dataset = GestureDataset(x_train, y_train)
val_dataset = GestureDataset(x_val, y_val)

# 创建 DataLoader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)


In [17]:
from torch import log_softmax


class GestureClassifier(pl.LightningModule):
    def __init__(self, input_dim, num_classes,learn_rate,weight_decay):
        super(GestureClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.TransformerEncoderLayer(d_model=32,nhead=8),
            nn.Linear(32, 64),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(64,64),
            nn.Dropout(0.1),
            nn.ReLU(),
            nn.Linear(64, num_classes),  # 输出 logits
            nn.LogSoftmax()
        )
        self.loss_fn = nn.CrossEntropyLoss()
        self.learn_rate = learn_rate
        self.weight_decay = weight_decay

        # 评估指标，指定 task 为 'multiclass'，并提供 num_classes
        self.accuracy = torchmetrics.Accuracy(task='multiclass', num_classes=num_classes)
        self.precision = torchmetrics.Precision(task='multiclass', num_classes=num_classes, average='weighted')
        self.recall = torchmetrics.Recall(task='multiclass', num_classes=num_classes, average='weighted')
        self.f1 = torchmetrics.F1Score(task='multiclass', num_classes=num_classes, average='weighted')

    def forward(self, x):
        x = self.model(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)
        
        # 记录训练损失
        self.log('train_loss', loss, prog_bar=True, logger=True)
        
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_fn(logits, y)

        # 计算预测类别
        preds = torch.argmax(logits, dim=1)

        # 更新评估指标
        acc = self.accuracy(preds, y)
        prec = self.precision(preds, y)
        rec = self.recall(preds, y)
        f1 = self.f1(preds, y)

        # 使用相同的 global_step 记录所有评估指标
        self.log('val_loss', loss, prog_bar=True, logger=True)
        self.log('val_acc', acc, prog_bar=True, logger=True)
        self.log('val_precision', prec, prog_bar=True, logger=True)
        self.log('val_recall', rec, prog_bar=True, logger=True)
        self.log('val_f1', f1, prog_bar=True, logger=True)
        
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.learn_rate,weight_decay=self.weight_decay)
        return optimizer

In [18]:

# 定义 TensorBoard logger
logger = TensorBoardLogger('tb_logs', name='gesture_classification')

# 获取输入维度和类别数量
input_dim = x_train.shape[1]
num_classes = len(label_encoder.classes_)

# 创建模型
model = GestureClassifier(input_dim=input_dim, num_classes=num_classes,learn_rate=learning_rate,weight_decay=weight_decay)

# 定义 Trainer，添加 TensorBoard logger
trainer = pl.Trainer(max_epochs=train_epochs, logger=logger,accelerator="gpu",devices=1)

# 训练模型
trainer.fit(model, train_loader, val_loader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                | Params | Mode 
----------------------------------------------------------
0 | model     | Sequential          | 154 K  | train
1 | loss_fn   | CrossEntropyLoss    | 0      | train
2 | accuracy  | MulticlassAccuracy  | 0      | train
3 | precision | MulticlassPrecision | 0      | train
4 | recall    | MulticlassRecall    | 0      | train
5 | f1        | MulticlassF1Score   | 0      | train
----------------------------------------------------------
154 K     Trainable params
0         Non-trainable params
154 K     Total params
0.618     Total estimated model params size (MB)
28        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

d:\Applications\Scoop\apps\anaconda3\current\App\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
d:\Applications\Scoop\apps\anaconda3\current\App\Lib\site-packages\torch\nn\modules\module.py:1553: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
d:\Applications\Scoop\apps\anaconda3\current\App\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=512` reached.


In [19]:
# 验证模型
trainer.validate(model, val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │    0.9716494679450989     │
│          val_f1           │    0.9720129370689392     │
│         val_loss          │    0.1327303797006607     │
│       val_precision       │    0.9800659418106079     │
│        val_recall         │    0.9716494679450989     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 0.1327303797006607,
  'val_acc': 0.9716494679450989,
  'val_precision': 0.9800659418106079,
  'val_recall': 0.9716494679450989,
  'val_f1': 0.9720129370689392}]

In [20]:
torch.save(model.state_dict,"checkpoint.pth")

In [21]:
dummy_input = torch.randn(1, input_dim)  # 输入维度应与模型的输入匹配
torch.onnx.export(model, dummy_input, "gesture_classifier.onnx", 
                  export_params=True,        # 导出所有权重
                  opset_version=14,          # ONNX 版本
                  do_constant_folding=True,  # 常量折叠优化
                  input_names=['input'],     # 输入名
                  output_names=['output'])   # 输出名

d:\Applications\Scoop\apps\anaconda3\current\App\Lib\site-packages\torch\nn\modules\module.py:1562: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return forward_call(*args, **kwargs)


In [22]:
label_encoder.classes_

array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'compute',
       'delete', 'div', 'minus', 'mul', 'natual', 'plus'], dtype=object)